In [1]:
import numpy as np
from math import sqrt

In [2]:
def norm(x):
    return sqrt(sum(t**2 for t in x))

def sqrnorm(x):
    return sum(t**2 for t in x)

def L2norm(vector):
    sumsquares = np.sum(np.power(vector,2))
    return sumsquares**(0.5)

In [3]:
def non_approximate_gradient(x, a):
    gradient = np.zeros(len(x))
    n = len(x)-1
#     print(n)
    for i in range(n-1):
#         print(i)
        # Special cases
        if i == 0:
            #print(a[n])
            gradient[i] = 2 * x[0] * (x[0] * x[1] - a[0]) + 2 * x[n] * (x[n] * x[0] - a[n])
        elif i == n:
        
        # Other
            gradient[n] = 2 * x[n-1] * ( x[n-1] * x[n] - a[n-1] ) + 2*x[1]*(x[n]*x[1] - a[n])
        else:
            gradient[i] = 2 * x[i-1] * (x[i-1] * x[i] - a[i-1]) + 2*x[i+1]*(x[i]*x[i+1] - a[i-1])
            
    return gradient

In [4]:
def armijo(f, grad_approx, x, c, alpha, stepdir, a):
    return (
        f(np.array(x - alpha*stepdir), a) < f(x, a) - c * alpha*( np.dot(stepdir, grad_approx) )
    )

In [5]:
def optimizer(f, initial, tol, a, max_iter, c, c1 ,c2):
    x_current = initial
    n = len(initial)
    acc_x = np.array([0]*len(initial))
    
    iteration = 0
    
    for i in range(max_iter):
        
        grad_approx = non_approximate_gradient(x_current,a)
        
        alpha = 1
        
        stepdir = c1*acc_x + c2*grad_approx
        
        while armijo(f, grad_approx, x_current, c, alpha * 1.5, stepdir, a):
            alpha *= 1.5
        max_iter_armijo = 100
        cur_iter_armijo = 0
        while not armijo(f, grad_approx, x_current, c, alpha, stepdir, a):
            alpha *= 0.5        
            cur_iter_armijo += 1
            if cur_iter_armijo > max_iter_armijo:
                return x_current
              
        x_next = list(x_current - alpha * stepdir)
        x_current = x_next   
        
        acc_x = stepdir
        iteration = iteration + 1
        
        print('alpha | grad ', alpha, grad_approx)
        
    return list(x_current)

# Enter sample input

In [6]:
def f(x, a):
    result = 0
    n = len(a)
    for i in range(n - 1):
        result += (x[i] * x[i + 1] - a[i]) ** 2
    result += (x[n - 1] * x[0] - a[n - 1]) ** 2
    return result

In [7]:
a = [2.9675555145514085, 0.6900184616445497, 2.9410424526610592]
f_threshold = 0.02

In [8]:
tol = f_threshold
initial = np.ones(len(a))

c , c1 , c2 =  [0.1, 0.9, 0.1]
max_iter = 100000


sol = optimizer(f, initial, tol, a, max_iter, c, c1 ,c2)

print(sol)

alpha | grad  3.375 [-7.81719593  0.          0.        ]
[3.638303627868416, 1.0, 1.0]


# Optimization for hyperparameters

In [9]:
# # INPUT FINAL COORDINATES!!
# # INPUT FINAL COORDINATES!!
# # INPUT FINAL COORDINATES!!
# print('CARE CARE CARE CARE')
# print('CARE CARE CARE CARE')
# print('CARE CARE CARE CARE')
# print('CARE CARE CARE CARE')
# print('The final coordinates the your require for optimization are:')
final_coord_vec = [3.55648, 0.83441, 0.82695]
# print(final_coord_vec)

#Other settings for hyperparameter optimization
div = 10
iter_max = 30

from itertools import product

c = np.linspace(0, 1, div, endpoint=False)[1:]
c1 = np.linspace(0, 1, div, endpoint=False)[1:]
c2 = np.linspace(0, 1, div, endpoint=False)[1:]

n_ele = len(c)*len(c1)*len(c2)
vec_hyperpara = []
for c, c1, c2 in product(c, c1, c2):
    vec_hyperpara.append([c, c1, c2])
    
list_of_error = []
list_of_norm = []
for p in range(n_ele):
    c_init, c1_init, c2_init = vec_hyperpara[p]  
    
    sol = optimizer(f, initial, tol, a, max_iter = iter_max, c = c_init, c1 = c1_init, c2 = c2_init)
    
    list_of_error.append( list(np.array(sol) - final_coord_vec) )
    list_of_norm.append( L2norm(list_of_error[p]) )
minpos = list_of_norm.index(min(list_of_norm))
best_hyperpara = vec_hyperpara[minpos]

# print('best hyperparameters || c , c1 , c2 = ', best_hyperpara)

c , c1 , c2 = best_hyperpara

# TEST 1 # 10,100 ##  c , c1 , c2 =  [0.30000000000000004, 0.30000000000000004, 0.1]
# TEST 1 # 20,100 ##  c , c1 , c2 =  [0.15000000000000002, 0.35000000000000003, 0.05]

#c , c1 , c2 =  [0.15000000000000002, 0.35000000000000003, 0.05]

alpha | grad  3.375 [-7.81719593  0.          0.        ]
alpha | grad  1.5 [6.27529294 0.         0.        ]
alpha | grad  3.375 [-1.11633594  0.          0.        ]
alpha | grad  2.25 [0.35799334 0.         0.        ]
alpha | grad  1.5 [-0.18107081  0.          0.        ]
alpha | grad  1.5 [-7.81719593  0.          0.        ]
alpha | grad  1 [3.33451842 0.         0.        ]
alpha | grad  1 [-0.96664359  0.          0.        ]
alpha | grad  3.375 [0.12712514 0.         0.        ]
alpha | grad  0.5 [-0.17131369  0.          0.        ]
alpha | grad  1 [-7.81719593  0.          0.        ]
alpha | grad  0.5 [3.33451842 0.         0.        ]
alpha | grad  0.125 [0.01089518 0.         0.        ]
alpha | grad  0.03125 [-0.06789811  0.          0.        ]
alpha | grad  0.001953125 [-0.06485056  0.          0.        ]
alpha | grad  6.103515625e-05 [-0.06453225  0.          0.        ]
alpha | grad  3.637978807091713e-12 [-0.06452195  0.          0.        ]
alpha | grad  1 [-7.8

alpha | grad  3.552713678800501e-15 [-0.50796615  0.          0.        ]
alpha | grad  1.5 [-0.50796615  0.          0.        ]
alpha | grad  1 [-7.81719593  0.          0.        ]
alpha | grad  2.220446049250313e-16 [3.33451842 0.         0.        ]
alpha | grad  1.5 [3.33451842 0.         0.        ]
alpha | grad  1.7763568394002505e-15 [-0.50796615  0.          0.        ]
alpha | grad  1 [-0.50796615  0.          0.        ]
alpha | grad  1 [-7.81719593  0.          0.        ]
alpha | grad  0.5 [11.94044071  0.          0.        ]
alpha | grad  0.015625 [0.22845477 0.         0.        ]
alpha | grad  0.00390625 [-0.0093404  0.         0.       ]
alpha | grad  0.0009765625 [-0.05635642  0.          0.        ]
alpha | grad  3.552713678800501e-15 [-0.06556922  0.          0.        ]
alpha | grad  0.5 [-7.81719593  0.          0.        ]
alpha | grad  0.5 [-7.81719593  0.          0.        ]
alpha | grad  1.1102230246251565e-16 [3.33451842 0.         0.        ]
alpha | grad

alpha | grad  0.25 [-7.81719593  0.          0.        ]
alpha | grad  0.0625 [-2.82706179  0.          0.        ]
alpha | grad  0.015625 [-0.47220385  0.          0.        ]
alpha | grad  0.0009765625 [-0.0996352  0.         0.       ]
alpha | grad  0.000244140625 [-0.0872318  0.         0.       ]
alpha | grad  1.52587890625e-05 [-0.08554668  0.          0.        ]
alpha | grad  7.62939453125e-06 [-0.0854858  0.         0.       ]
alpha | grad  3.814697265625e-06 [-0.08546648  0.          0.        ]
alpha | grad  1.9073486328125e-06 [-0.0854596  0.         0.       ]
alpha | grad  1.862645149230957e-09 [-0.08545685  0.          0.        ]
alpha | grad  1.7763568394002505e-15 [-0.08545685  0.          0.        ]
alpha | grad  0.25 [-7.81719593  0.          0.        ]
alpha | grad  0.03125 [-1.51582381  0.          0.        ]
alpha | grad  0.015625 [-0.38602861  0.          0.        ]
alpha | grad  3.375 [-7.81719593  0.          0.        ]
alpha | grad  3.375 [6.27529294 0. 

alpha | grad  0.000244140625 [-0.12679951  0.          0.        ]
alpha | grad  0.000244140625 [-0.12658333  0.          0.        ]
alpha | grad  6.103515625e-05 [-0.12636778  0.          0.        ]
alpha | grad  1.52587890625e-05 [-0.12631399  0.          0.        ]
alpha | grad  3.814697265625e-06 [-0.12630055  0.          0.        ]
alpha | grad  9.5367431640625e-07 [-0.12629719  0.          0.        ]
alpha | grad  9.5367431640625e-07 [-0.12629635  0.          0.        ]
alpha | grad  2.384185791015625e-07 [-0.12629551  0.          0.        ]
alpha | grad  2.9802322387695312e-08 [-0.1262953  0.         0.       ]
alpha | grad  2.9802322387695312e-08 [-0.12629527  0.          0.        ]
alpha | grad  0.25 [-7.81719593  0.          0.        ]
alpha | grad  0.0625 [-1.51582381  0.          0.        ]
alpha | grad  0.0625 [-0.56479089  0.          0.        ]
alpha | grad  0.03125 [-0.22035409  0.          0.        ]
alpha | grad  0.015625 [-0.15447439  0.          0.      

alpha | grad  0.125 [3.33451842 0.         0.        ]
alpha | grad  0.0625 [1.28012112 0.         0.        ]
alpha | grad  0.03125 [0.40521246 0.         0.        ]
alpha | grad  0.03125 [0.1326863 0.        0.       ]
alpha | grad  0.03125 [-0.02151478  0.          0.        ]
alpha | grad  0.03125 [-0.09487855  0.          0.        ]
alpha | grad  0.25 [-7.81719593  0.          0.        ]
alpha | grad  0.0625 [-3.98552839  0.          0.        ]
alpha | grad  0.03125 [-1.83469298  0.          0.        ]
alpha | grad  0.03125 [-0.92768991  0.          0.        ]
alpha | grad  0.0078125 [-0.29357434  0.          0.        ]
alpha | grad  0.0078125 [-0.19981924  0.          0.        ]
alpha | grad  0.001953125 [-0.14416451  0.          0.        ]
alpha | grad  0.001953125 [-0.13564836  0.          0.        ]
alpha | grad  0.0009765625 [-0.12993416  0.          0.        ]
alpha | grad  0.00048828125 [-0.12780863  0.          0.        ]
alpha | grad  0.000244140625 [-0.126934

...

alpha | grad  2.3283064365386963e-10 [-6.32740817  0.          0.        ]
alpha | grad  2.9103830456733704e-11 [-6.32740816  0.          0.        ]
alpha | grad  4.440892098500626e-16 [-6.32740815  0.          0.        ]
alpha | grad  4.547473508864641e-13 [-6.32740815  0.          0.        ]
alpha | grad  7.275957614183426e-12 [-6.32740815  0.          0.        ]
alpha | grad  2.9103830456733704e-11 [-6.32740815  0.          0.        ]
alpha | grad  1.1641532182693481e-10 [-6.32740815  0.          0.        ]
alpha | grad  2.3283064365386963e-10 [-6.32740815  0.          0.        ]
alpha | grad  2.2737367544323206e-13 [-6.32740814  0.          0.        ]
alpha | grad  1.1368683772161603e-13 [-6.32740814  0.          0.        ]
alpha | grad  4.440892098500626e-16 [-6.32740814  0.          0.        ]
alpha | grad  1.4551915228366852e-11 [-6.32740814  0.          0.        ]
alpha | grad  2.3283064365386963e-10 [-6.32740813  0.          0.        ]
alpha | grad  4.6566128730773

alpha | grad  1.1641532182693481e-10 [-6.32740818  0.          0.        ]
alpha | grad  3.637978807091713e-12 [-6.32740817  0.          0.        ]
alpha | grad  2.2737367544323206e-13 [-6.32740817  0.          0.        ]
alpha | grad  2.2737367544323206e-13 [-6.32740817  0.          0.        ]
alpha | grad  0.25 [-7.81719593  0.          0.        ]
alpha | grad  5.551115123125783e-17 [-5.84414745  0.          0.        ]
alpha | grad  1.3877787807814457e-17 [-5.84414745  0.          0.        ]
alpha | grad  1.3877787807814457e-17 [-5.84414745  0.          0.        ]
alpha | grad  2.7755575615628914e-17 [-5.84414745  0.          0.        ]
alpha | grad  5.551115123125783e-17 [-5.84414745  0.          0.        ]
alpha | grad  0.125 [-7.81719593  0.          0.        ]
alpha | grad  0.015625 [-7.09168099  0.          0.        ]
alpha | grad  0.0078125 [-6.73740738  0.          0.        ]
alpha | grad  0.001953125 [-6.45272557  0.          0.        ]
alpha | grad  0.0002441406

alpha | grad  0.0009765625 [-7.32031363  0.          0.        ]
alpha | grad  0.0001220703125 [-7.29849639  0.          0.        ]
alpha | grad  6.103515625e-05 [-7.29566168  0.          0.        ]
alpha | grad  7.62939453125e-06 [-7.29422304  0.          0.        ]
alpha | grad  9.5367431640625e-07 [-7.29404194  0.          0.        ]
alpha | grad  4.76837158203125e-07 [-7.29401922  0.          0.        ]
alpha | grad  2.9103830456733704e-11 [-7.29400785  0.          0.        ]
alpha | grad  4.547473508864641e-13 [-7.29400785  0.          0.        ]
alpha | grad  0.0625 [-7.81719593  0.          0.        ]
alpha | grad  0.00390625 [-7.40196058  0.          0.        ]
alpha | grad  0.0009765625 [-7.32627894  0.          0.        ]
alpha | grad  0.000244140625 [-7.30339554  0.          0.        ]
alpha | grad  0.0001220703125 [-7.29722904  0.          0.        ]
alpha | grad  9.5367431640625e-07 [-7.29404203  0.          0.        ]
alpha | grad  2.384185791015625e-07 [-7.2

alpha | grad  0.25 [-7.81719593  0.          0.        ]
alpha | grad  0.03125 [-7.728129  0.        0.      ]
alpha | grad  0.125 [-7.81719593  0.          0.        ]
alpha | grad  0.015625 [-7.728129  0.        0.      ]
alpha | grad  0.125 [-7.81719593  0.          0.        ]
alpha | grad  2.220446049250313e-16 [-7.62630626  0.          0.        ]
alpha | grad  0.0625 [-7.81719593  0.          0.        ]
alpha | grad  0.0078125 [-7.728129  0.        0.      ]
alpha | grad  0.0625 [-7.81719593  0.          0.        ]
alpha | grad  4.440892098500626e-16 [-7.68199174  0.          0.        ]
alpha | grad  0.0625 [-7.81719593  0.          0.        ]
alpha | grad  1.1102230246251565e-16 [-7.62630626  0.          0.        ]
alpha | grad  0.0625 [-7.81719593  0.          0.        ]
alpha | grad  5.551115123125783e-17 [-7.56107258  0.          0.        ]
alpha | grad  0.03125 [-7.81719593  0.          0.        ]
alpha | grad  0.00390625 [-7.728129  0.        0.      ]
alpha | grad

alpha | grad  0.0625 [-7.81719593  0.          0.        ]
alpha | grad  5.551115123125783e-17 [-7.56107258  0.          0.        ]
alpha | grad  5.551115123125783e-17 [-7.56107258  0.          0.        ]
alpha | grad  0.03125 [-7.81719593  0.          0.        ]
alpha | grad  0.001953125 [-7.728129  0.        0.      ]
alpha | grad  0.0001220703125 [-7.70753504  0.          0.        ]
alpha | grad  5.684341886080802e-14 [-7.70560167  0.          0.        ]
alpha | grad  2.2737367544323206e-13 [-7.70560167  0.          0.        ]
alpha | grad  0.03125 [-7.81719593  0.          0.        ]
alpha | grad  6.103515625e-05 [-7.70625389  0.          0.        ]
alpha | grad  1.4210854715202004e-14 [-7.70548788  0.          0.        ]
alpha | grad  7.105427357601002e-15 [-7.70548788  0.          0.        ]


# Print final solution

In [10]:
sol = optimizer(f, initial, tol, a, max_iter=10000, c = c, c1 = c1 , c2 = c2) 
print(*sol)

alpha | grad  3.375 [-7.81719593  0.          0.        ]
3.638303627868416 1.0 1.0
